# 1. Abstract  

K-means is one of the most common data processing algorithms. The algorithms obtain an excellent final solution by employing proper initialization. However, from a theoretical perspective concerning quality and efficiency, k-means is not a suitable clustering algorithm. This is because in the worst case it produces a final solution which is locally optimal, although it doesn't meet the typical global optimum. Besides, this paper will discuss the keys that were applied in the process improving k-means from traditional to the current modern algorithm. The big step taken was to develop the initialization procedure of traditional k-means which changed the performance of Lloyd's iteration. The improvement resulted in k-means++ algorithm which improves upon the running time of iteration. The algorithm picks only the first cluster center consistently at random from the data and probability is used in selecting the subsequent cluster center. Initialization of k-means++ leads to an O(log K) which is an approximation of the solution optimum. However, K-means++ algorithm has its downside which is being intrinsically sequential. Although it has a total running of 0(nkd), its k-clustering is similar to that of a single Lloyd's iteration. The downside led to obtainment a parallel version of k-means++ known as k-means||. The k-means|| algorithm uses the idea of sampling O(k) points per round, and the process is repeated to O(log n) rounds. Moreover, at the end the algorithm only O(k log n) points are left, which form a solution in a constant factor.

# 2. Background

In data mining and processing, some specifics algorithms are applied to cluster or group the data to meaningful information. Those algorithms are referred as clustering algorithms. In this case, scalable k-means paper is used in the research about of k-means algorithms, their efficiency in operation and the solution they offer in data mining processes. K-means clustering algorithms are used when data is unbalanced that is data without defined groups or categories (Bahmani, Moseley, Vattani, Kumar & Vassilvitskii, 2012). These algorithms aim to group data into clusters whereby variable k represents the number of groups or cluster. Besides, the algorithm works iteratively to allocate every data point to a single k group by use of provided attributes. It should be noted that data points are clustered on the ground of feature similarity. K-means results in centroids of the k clusters that are used in new data labeling and markers for the training data.

# 3. Application of the Algorithm

The k-means algorithm can be useful in almost all the fields. The algorithm is used in data clustering applications in different areas (Bahmani, Moseley, Vattani, Kumar & Vassilvitskii, 2012). For instance, clustering helps marketers advance their customers base as well as work on the goal areas. The k-means help group people using different criteria including purchasing power and willingness. The grouping is on the basis of their similarity in several ways which are related to the product under deliberation. In addition, clustering can be used to study earthquake. K-means analyses shows the next likely location where an earthquake can transpire by use of available data on the areas hit by the quake in a region. Moreover, k-means algorithm is mostly applied in large database management systems whereby data is mined, cleaned and analyzed for useful information. The advantage of k-means is that it is computationally faster than other clustering algorithms such as hierarchical clustering algorithm. The main disadvantage is that k-means cannot grip non-globular data of diverse sizes and densities.

# 4. Implementation 

In [8]:
import numpy as np
import pandas as pd
import scipy.spatial.distance as dist
import collections 

## 4.1 Kmeans

### 4.1.1 Algorithm

**Initialize cluster centroids $\mu_1, \mu_2....\mu_k \in \mathbb{R}^n$ randomly** <br>
Repeat until convergence:{<br> 
+ For every i, set <br>
> $c^{(i)}:= argmin||x^{(i)} - u_j||^2$  <br>
+ For every j, set<br>
> $\mu_j := \frac{\sum_{i=1}^{m} 1 \{c^{(i) = j}\}x^{(i)}}{\sum_{i=1}^{m} 1\{c^{(i) = j}\}}$<br>

}

### 4.1.2 Code

In [9]:
def kmeans(data, k, centroids, max_iter=10000):
    
    for i in range(max_iter):
        cdist = (dist.cdist(data, centroids))**2
        labels = np.argmin(cdist, axis=1)
        update_centroids = np.zeros(centroids.shape)
        for j in range(k):
            # check if the centroid is the closest to some data point
            if sum(labels == j) != 0:
                update_centroids[j] = np.mean(data[labels ==j], axis=0)
            else:
                # if not, leave the lone centroid unmoved
                update_centroids[j] = centroids[j]
                
        if np.allclose(update_centroids, centroids):
            print("Algorithm converged after", i, "iterations.")
            return centroids
        else:
            centroids = update_centroids
        
    print("Warning: maximum number of iterations reached. Failed to converge.")
    return centroids

#### Random initial centroids

In [10]:
def main():
    k, n, d = 20, 1000, 15
    
    mean, cov = np.zeros(d), np.eye(d)
    data = np.random.multivariate_normal(mean, cov, n)
    #random initial 
    initial_centers = np.random.multivariate_normal(mean, cov, k)
    
    centroids = kmeans(data, k, initial_centers)

if __name__ == '__main__':
    main()

Algorithm converged after 21 iterations.


## 4.2 Kmeans++

### 4.2.1 Algorithm

**Choose the centers one by onte in a controlled fasion, where the current set of chosen centers will stochastically bias the choice of the next center**<br>

First sample a point uniformly at random from X : C<br>
Set how many clusters are needed: k <br>
Distance between points and centers: $d^2(x, C)$<br>
Compute the cost of X: $\phi x(C) = \sum_{x\in X} min_{i=1,..k} ||y - c_i||^2$<br>
While |C| < k do:
+ Sample x $\in$ X with probability $p_x = \frac{d^2(x, C)}{\phi x(C)}$ <br>
+ Stack C with {x}: C = C $\cup$ {x} <br> 

end while

### 4.2.1 Code

In [11]:
def kmeans_pp(weights, data, k):
    first_random = np.random.choice(data.shape[0], 1)
    C = data[first_random, :]
    
    for i in range(k-1):
        cdist = (dist.cdist(data, C))**2
        cdist_min = np.min(cdist, axis = 1)* weights
        prob = cdist_min/np.sum(cdist_min)
        new_center = np.random.choice(data.shape[0],1, p=prob)
        C = np.vstack([C, data[new_center,:]])
        
    return C

#### Equal weights for kmenas++

In [12]:
def main():
    k, n, d = 20, 1000, 15
    
    mean, cov = np.zeros(d), np.eye(d)
    data = np.random.multivariate_normal(mean, cov, n)
    #equal weights for kmenas++
    initial_centers = kmeans_pp(1, data, 20)
    
    centroids = kmeans(data, k, initial_centers)

if __name__ == '__main__':
    main()

Algorithm converged after 16 iterations.


## 4.3 Kmeans_II

### 4.3.1 Algorithm

**Parallel version of Kmeans++**<br>

First set an oversampling factor l = $\Omega(K)$<br>
Sample a point uniformly at random from X : C<br>
Set how many clusters are needed: k <br>
Distance between points and centers: $d^2(x, C)$<br>
Compute the cost of X: $\phi x(C) = \sum_{x\in X} min_{i=1,..k} ||y - c_i||^2$<br>

for O(log($\phi$)) times do:
+ Sample each point x $\in$ X independently with probability $p_x = \frac{l * d^2(x, C)}{\phi x(C)}$ <br>
+ Stack C with {x}: C = C $\cup$ {x} <br> 

end for<br>
For each point x$\in$ C, compute $w_x$ to be the number of points in X closer than other points in C<br>
Recluster the weighted points in C into k clusters. 

### 4.3.2 Code

#### Compute each weight

In [13]:
def get_weight(C, data): 
    weights=np.zeros(C.shape[0])
    cdist = (dist.cdist(data,C))**2
    min_cdist = np.argmin(cdist, axis = 1)
    count = collections.Counter(min_cdist) 
    weights = list(collections.OrderedDict(sorted(count.items(), key=lambda x: x[0])).values())
    weights=np.array(weights)/sum(weights)
    return weights

In [ ]:
def kmeans_II(data, k, l, max_iter=10000):
    first_random = np.random.choice(data.shape[0], 1)
    C = data[first_random, :]
    
    cdist = (dist.cdist(data, C))**2
    cdist_min = np.min(cdist, axis = 1)
    cost_phi = np.sum(cdist_min)
    
    for i in range(int(round(np.log(cost_phi)))):
        cdist = (dist.cdist(data, C))**2
        cdist_min = np.min(cdist, axis = 1)
        prob = cdist_min * l/np.sum(cdist_min)
        for j in range(data.shape[0]):
            if np.random.uniform() <= prob[j] and data[j,:] not in C:
                C = np.vstack([C, data[j,:]])
   
    weights= get_weight(C, data)

    return kmeans_pp(weights, C,k)
    

#### Initial centroids with weights

In [15]:
def main():
    k, n, d = 20, 1000, 15
    
    mean, cov = np.zeros(d), np.eye(d)
    data = np.random.multivariate_normal(mean, cov, n)
    #initial with weight 
    initial_centers = kmeans_II(data, 20, 10)
    
    centroids = kmeans(data, k, initial_centers)

if __name__ == '__main__':
    main()

Algorithm converged after 20 iterations.
